# 10.1GroupBy

In [1]:
import pandas as pd
import numpy as np
df=pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                'key2':['one', 'two', 'one', 'two', 'one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.598483,0.005175,a,one
1,-0.131168,-0.096328,a,two
2,-0.883486,1.294931,b,one
3,1.895866,1.708512,b,two
4,-0.677008,1.900579,a,one


按key1进行分组，并计算data1的平均值：可以先访问data1，再根据key1调用groupby

In [2]:
grouped=df['data1'].groupby(df['key1'])

变量grouped是一个GroupBy对象。它实际上还没有进行任何计算，只是含有一些有关分组键df['key1']的中间数据而已。换句话说，该对象已经有了接下来对各分组执行运算所需的一切信息。例如，我们可以调用GroupBy的mean方法来计算分组平均值：

In [3]:
grouped.mean()

key1
a   -0.069898
b    0.506190
Name: data1, dtype: float64

如果我们一次传入多个数组的列表，就会得到不同的结果：

In [4]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.039262
      two    -0.131168
b     one    -0.883486
      two     1.895866
Name: data1, dtype: float64

上面通过两个键对数组进行分组，得到一个具有层次化索引的Series

In [5]:
means.unstack()

key2,one,two
key1,,
a,-0.039262,-0.131168
b,-0.883486,1.895866


分组键可以是任何长度适当的数组：

In [6]:
states=np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years=np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states,years]).mean()

California  2005   -0.131168
            2006   -0.883486
Ohio        2005    1.247175
            2006   -0.677008
Name: data1, dtype: float64

size方法可以返回一个含有分组大小的Series：

In [7]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

通常分组信息就位于相同的要处理DataFrame中。这里，你还可以将列名（可以是字符串、数字或其他Python对象）用作分组键：

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.069898,0.603142
b,0.506190,1.501721


In [9]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.039262  0.952877
     two  -0.131168 -0.096328
b    one  -0.883486  1.294931
     two   1.895866  1.708512

## 对分组进行迭代
GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）

In [10]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  0.598483  0.005175    a  one
1 -0.131168 -0.096328    a  two
4 -0.677008  1.900579    a  one
b
      data1     data2 key1 key2
2 -0.883486  1.294931    b  one
3  1.895866  1.708512    b  two


对于多重键的情况，元组的第一个元素将会是由键值组成的元组：

In [11]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0  0.598483  0.005175    a  one
4 -0.677008  1.900579    a  one
('a', 'two')
      data1     data2 key1 key2
1 -0.131168 -0.096328    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.883486  1.294931    b  one
('b', 'two')
      data1     data2 key1 key2
3  1.895866  1.708512    b  two


可以对这些数据片段做任何操作

In [12]:
grouped=df.groupby(['key1'])
group_lst=list(grouped)
group_lst

[('a',       data1     data2 key1 key2
  0  0.598483  0.005175    a  one
  1 -0.131168 -0.096328    a  two
  4 -0.677008  1.900579    a  one), ('b',       data1     data2 key1 key2
  2 -0.883486  1.294931    b  one
  3  1.895866  1.708512    b  two)]

In [13]:
group_dict=dict(group_lst)
group_dict

{'a':       data1     data2 key1 key2
 0  0.598483  0.005175    a  one
 1 -0.131168 -0.096328    a  two
 4 -0.677008  1.900579    a  one, 'b':       data1     data2 key1 key2
 2 -0.883486  1.294931    b  one
 3  1.895866  1.708512    b  two}

groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组。拿上面例子中的df来说，我们可以根据dtype对列进行分组：

In [14]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [15]:
grouped=df.groupby(df.dtypes,axis=1)
for name,group in grouped:
    print(name)
    print(group)

float64
      data1     data2
0  0.598483  0.005175
1 -0.131168 -0.096328
2 -0.883486  1.294931
3  1.895866  1.708512
4 -0.677008  1.900579
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 选取一列或列的子集
对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。也就是说：

In [16]:
df.groupby('key1')['data1']

是以下代码的语法糖

In [17]:
df['data1'].groupby(df['key1'])

尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，如果只需计算data2列的平均值并以DataFrame形式得到结果，可以这样写：

In [18]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.952877
     two  -0.096328
b    one   1.294931
     two   1.708512

上面这种方式得到的是DataFrame,下面的方式将得到一个Series:

In [19]:
df.groupby(['key1','key2'])['data2'].mean()

key1  key2
a     one     0.952877
      two    -0.096328
b     one     1.294931
      two     1.708512
Name: data2, dtype: float64

## 通过字典或Series进行分组
除数组以外，分组信息还可以其他形式存在。

In [20]:
people=pd.DataFrame(np.random.randn(5,5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,1.652782,-2.126197,-0.048948,-0.606249,-0.167628
Steve,-1.437960,1.431493,0.054133,0.616591,-0.464998
Wes,-0.226094,1.091871,0.282148,0.648325,0.816788
Jim,0.405687,1.097609,0.184327,-1.685966,0.872587
Travis,1.105123,-1.717144,-0.191068,-0.281411,-0.285878


In [21]:
maping={'a':'red','b':'red','c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}

可以将这个字典传给groupby

In [22]:
people.groupby(maping,axis=1).sum()

,blue,red
Joe,-0.655198,-0.641043
Steve,0.670723,-0.471465
Wes,0.930473,1.682565
Jim,-1.501639,2.375882
Travis,-0.472479,-0.897899


Series也有同样的功能，它可以被看做一个固定大小的映射：

In [23]:
map_series=pd.Series(maping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series,axis=1).sum()

,blue,red
Joe,-0.655198,-0.641043
Steve,0.670723,-0.471465
Wes,0.930473,1.682565
Jim,-1.501639,2.375882
Travis,-0.472479,-0.897899


In [25]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,2,3
Jim,2,3
Travis,2,3


## 通过函数进行分组
比起使用字典或Series，使用Python函数是一种更原生的方法定义分组映射。任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。具体点说，以上一小节的示例DataFrame为例，其索引值为人的名字。你可以计算一个字符串长度的数组，更简单的方法是传入len函数：

In [26]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.832375,0.063282,0.417527,-1.643891,1.521746
5,-1.437960,1.431493,0.054133,0.616591,-0.464998
6,1.105123,-1.717144,-0.191068,-0.281411,-0.285878


将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西在内部都会被转换为数组：

In [27]:
k_list=['one','one','one','two','two']
people.groupby([len,k_list]).min()

a         b         c         d         e
3 one -0.226094 -2.126197 -0.048948 -0.606249 -0.167628
  two  0.405687  1.097609  0.184327 -1.685966  0.872587
5 one -1.437960  1.431493  0.054133  0.616591 -0.464998
6 two  1.105123 -1.717144 -0.191068 -0.281411 -0.285878

## 根据索引级别分组
层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：

In [28]:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['city','tenor'])
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['city', 'tenor'])

In [29]:
mult_df=pd.DataFrame(np.random.randn(4,5),columns=columns)
mult_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.388490  0.085578 -0.492663  0.439164  0.125767
1      0.763793  0.961113  0.697277  0.059472  0.278217
2      0.486340  0.414555  0.003277  2.876365 -1.189215
3     -0.811461  1.332387  0.063102 -0.656239 -0.851543

要根据级别分组，使用level关键字传递级别序号或名字：

In [30]:
mult_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2数据聚合
你可以使用自己发明的聚合运算，还可以调用分组对象上已经定义好的任何方法。例如，quantile可以计算Series或DataFrame列的样本分位数。

In [31]:
df

,data1,data2,key1,key2
0,0.598483,0.005175,a,one
1,-0.131168,-0.096328,a,two
2,-0.883486,1.294931,b,one
3,1.895866,1.708512,b,two
4,-0.677008,1.900579,a,one


In [32]:
grouped=df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.452553
b    1.617931
Name: data1, dtype: float64

如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法即可：

In [33]:
def max_to_min(arr):
    return arr.max()-arr.min()
grouped.agg(max_to_min)

,data1,data2
key1,,
a,1.275491,1.996907
b,2.779352,0.413581


有些方法（如describe）也是可以用在这里的，即使严格来讲，它们并非聚合运算：

In [34]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.069898  0.639949 -0.677008 -0.404088 -0.131168  0.233657   
b      2.0  0.506190  1.965298 -0.883486 -0.188648  0.506190  1.201028   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.598483   3.0  0.603142  1.124759 -0.096328 -0.045576  0.005175   
b     1.895866   2.0  1.501721  0.292446  1.294931  1.398326  1.501721   

                          
           75%       max  
key1                      
a     0.952877  1.900579  
b     1.605117  1.708512

## 面向列的多函数应用

In [35]:
tips=pd.read_csv(r'E:\python\data\tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [36]:
tips['tip_pct']=tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


你可能希望对不同的列使用不同的聚合函数，或一次应用多个函数。

In [37]:
grouped=tips.groupby(['day','smoker'])
grouped_pct=grouped['tip_pct']
#可以将函数名以字符串的形式传入
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名：

In [38]:
grouped_pct.agg(['std','mean',max_to_min])

std      mean  max_to_min
day  smoker                                
Fri  No      0.028123  0.151650    0.067349
     Yes     0.051293  0.174783    0.159925
Sat  No      0.039767  0.158048    0.235193
     Yes     0.061375  0.147906    0.290095
Sun  No      0.042347  0.160113    0.193226
     Yes     0.154134  0.187250    0.644685
Thur No      0.038774  0.160298    0.193350
     Yes     0.039389  0.163863    0.151240

如果传入的是一个由(name,function)元组组成的列表，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序映射）：

In [39]:
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

想要对一个列或不同的列应用不同的函数。具体的办法是向agg传入一个从列名映射到函数的字典：

In [40]:
grouped.agg({'tip':'max','size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [41]:
grouped.agg({'tip':['min','max'],'size':'sum'})

tip        size
              min    max  sum
day  smoker                  
Fri  No      1.50   3.50    9
     Yes     1.00   4.73   31
Sat  No      1.00   9.00  115
     Yes     1.00  10.00  104
Sun  No      1.01   6.00  167
     Yes     1.50   6.50   49
Thur No      1.25   6.70  112
     Yes     2.00   5.00   40

## 以“没有行索引”的形式返回聚合数据
到目前为止，所有示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层次化的）。由于并不总是需要如此，所以你可以向groupby传入as_index=False以禁用该功能：

In [42]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 apply
回到之前那个小费数据集，假设你想要根据分组选出最高的5个tip_pct值。首先，编写一个选取指定列具有最大值的行的函数：

In [43]:
def topn(df,n=5,column='tip_pct'):
    return df.sort_values(by=column,ascending=False)[0:n]
topn(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


现在，如果对smoker分组并用该函数调用apply，就会得到：

In [44]:
tips.groupby('smoker').apply(topn)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

如果传给apply的函数能够接受其他参数或关键字，则可以将这些内容放在函数名后面一并传入：

In [45]:
tips.groupby(['smoker','day']).apply(topn,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

## 禁止分组键
从上面的例子中可以看出，分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将group_keys=False传入groupby即可禁止该效果：

In [46]:
tips.groupby('smoker',group_keys=False).apply(topn)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


## 分位数和桶分析
pandas有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如cut和qcut）。将这些函数跟groupby结合起来，就能非常轻松地实现对数据集的桶（bucket）或分位数（quantile）分析了。以下面这个简单的随机数据集为例，我们利用cut将其装入长度相等的桶中：

In [47]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                   'data2':np.random.randn(1000)})
quartiles=pd.cut(frame['data1'],4)
quartiles.head()

0     (-0.625, 1.033]
1    (-2.282, -0.625]
2     (-0.625, 1.033]
3     (-0.625, 1.033]
4     (-0.625, 1.033]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.947, -2.282] < (-2.282, -0.625] < (-0.625, 1.033] < (1.033, 2.691]]

由cut返回的Categorical对象可直接传递到groupby。因此，我们可以像下面这样对data2列做一些统计计算：

In [48]:
def stats(group):
    return {'min':group.mean(),'max':group.max(),'count':group.count(),'mean':group.mean()}
grouped=frame.groupby(quartiles)['data2'].apply(stats)
grouped

data1                  
(-3.947, -2.282]  count     13.000000
                  max        1.174266
                  mean       0.174215
                  min        0.174215
(-2.282, -0.625]  count    252.000000
                  max        2.578780
                  mean       0.072863
                  min        0.072863
(-0.625, 1.033]   count    572.000000
                  max        2.951420
                  mean       0.031708
                  min        0.031708
(1.033, 2.691]    count    163.000000
                  max        3.204751
                  mean       0.079814
                  min        0.079814
Name: data2, dtype: float64

In [49]:
grouped.unstack()

,count,max,mean,min
data1,,,,
"(-3.947, -2.282]",13.0,1.174266,0.174215,0.174215
"(-2.282, -0.625]",252.0,2.578780,0.072863,0.072863
"(-0.625, 1.033]",572.0,2.951420,0.031708,0.031708
"(1.033, 2.691]",163.0,3.204751,0.079814,0.079814


这些都是长度相等的桶。要根据样本分位数得到大小相等的桶，使用qcut即可。

In [50]:
quarting=pd.qcut(frame['data1'],5)
quarting.head()

0      (0.286, 0.881]
1    (-3.941, -0.845]
2      (0.286, 0.881]
3     (-0.228, 0.286]
4      (0.286, 0.881]
Name: data1, dtype: category
Categories (5, interval[float64]): [(-3.941, -0.845] < (-0.845, -0.228] < (-0.228, 0.286] < (0.286, 0.881] < (0.881, 2.691]]

In [51]:
frame.groupby(quarting)['data1'].apply(stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.941, -0.845]",200.0,-0.846609,-1.434173,-1.434173
"(-0.845, -0.228]",200.0,-0.229728,-0.509617,-0.509617
"(-0.228, 0.286]",200.0,0.286182,0.038629,0.038629
"(0.286, 0.881]",200.0,0.880128,0.563444,0.563444
"(0.881, 2.691]",200.0,2.691233,1.410677,1.410677


## 用特定于分组的值填充缺失值
对于缺失数据的清理工作，有时你会用dropna将其替换掉，而有时则可能会希望用一个固定值或由数据集本身所衍生出来的值去填充NA值。这时就得使用fillna这个工具了

In [52]:
s=pd.Series(np.random.randn(6))
s[::2]=np.NAN
s

0         NaN
1   -2.363263
2         NaN
3    0.644525
4         NaN
5   -0.523887
dtype: float64

In [53]:
s.fillna(s.mean())

0   -0.747542
1   -2.363263
2   -0.747542
3    0.644525
4   -0.747542
5   -0.523887
dtype: float64

假设你需要对不同的分组填充不同的值。一种方法是将数据分组，并使用apply和一个能够对各数据块调用fillna的函数即可。下面是一些有关美国几个州的示例数据，这些州又被分为东部和西部：

In [54]:
states=['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']
data=pd.Series(np.random.randn(8),index=states)
group_key=['East']*4+['West']*4
data

Ohio         -1.065487
New York     -1.418678
Vermont       1.783882
Florida      -0.352644
Oregon       -0.209531
Nevada       -0.220858
California   -0.449881
Idaho         0.553399
dtype: float64

将一些值设为缺失：

In [55]:
data['Vermont', 'Nevada', 'Idaho']=np.nan
data

Ohio         -1.065487
New York     -1.418678
Vermont            NaN
Florida      -0.352644
Oregon       -0.209531
Nevada             NaN
California   -0.449881
Idaho              NaN
dtype: float64

In [56]:
data.groupby(group_key).mean()

East   -0.945603
West   -0.329706
dtype: float64

我们可以用分组平均值去填充NA值:

In [57]:
fill_nan=lambda x : x.fillna(x.mean())
data.groupby(group_key).apply(fill_nan)

Ohio         -1.065487
New York     -1.418678
Vermont      -0.945603
Florida      -0.352644
Oregon       -0.209531
Nevada       -0.329706
California   -0.449881
Idaho        -0.329706
dtype: float64

也可以在代码中预定义各组的填充值。由于分组具有一个name属性，所以我们可以拿来用一下：

In [58]:
fill_value={'East':0.5,'West':1}
fill_func=lambda x :x.fillna(fill_value[x.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.065487
New York     -1.418678
Vermont       0.500000
Florida      -0.352644
Oregon       -0.209531
Nevada        1.000000
California   -0.449881
Idaho         1.000000
dtype: float64

## 随机采样和排列
假设你想要从一个大数据集中随机抽取（进行替换或不替换）样本以进行蒙特卡罗模拟（Monte Carlo simulation）或其他分析工作。“抽取”的方式有很多，这里使用的方法是对Series使用sample方法：

In [59]:
suits=['H', 'S', 'C', 'D']
card_cal=(list(range(1,11))+[10]*3)*4
base_name=['A']+list(range(2,11))+['J','K','Q']
cards=[]
for suit in suits:
    cards.extend(str(num)+suit for num in base_name)
deck=pd.Series(card_cal,index=cards)
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

现在，从整副牌中抽出5张，代码如下：

In [60]:
deck.sample(5)

5D     5
9C     9
AD     1
KD    10
7D     7
dtype: int64

In [61]:
def get_sample(deck,n=2):
    return deck.sample(n)
get_suit=lambda card :card[-1]
deck.groupby(get_suit).apply(get_sample)

C  QC     10
   3C      3
D  9D      9
   10D    10
H  8H      8
   5H      5
S  3S      3
   7S      7
dtype: int64

## 分组加权平均数和相关系数
根据groupby的“拆分－应用－合并”范式，可以进行DataFrame的列与列之间或两个Series之间的运算（比如分组加权平均）。以下面这个数据集为例，它含有分组键、值以及一些权重值：

In [62]:
df=pd.DataFrame({'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'], 'data': np.random.randn(8),'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.977147,0.733944
1,a,-1.326919,0.261830
2,a,-1.195815,0.011504
3,a,0.167074,0.481694
4,b,-0.217414,0.108749
5,b,0.456921,0.563473
6,b,0.551194,0.993203
7,b,0.889825,0.581737


In [63]:
get_wavg=lambda x :np.average(x['data'],weights=x['weights'])
grouped=df.groupby('category')
grouped.apply(get_wavg)

category
a   -0.670178
b    0.578023
dtype: float64

In [64]:
df2=df[['data','weights']]
df2.head()

,data,weights
0,-0.977147,0.733944
1,-1.326919,0.261830
2,-1.195815,0.011504
3,0.167074,0.481694
4,-0.217414,0.108749


另一个例子，考虑一个来自Yahoo!Finance的数据集，其中含有几只股票和标准普尔500指数（符号SPX）的收盘价：

In [65]:
close_px=pd.read_csv(r'E:\python\data\stock_px_2.csv',index_col=0,parse_dates=True)
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


使用pct_change计算close_px的百分比变化：

In [66]:
pct_px=close_px.pct_change().dropna()
pct_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [67]:
px_corr=lambda x :x.corrwith(x['SPX'])
get_year=lambda x :x.year
pct_px.groupby(get_year).apply(px_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


当然，你还可以计算列与列之间的相关系数。这里，我们计算Apple和Microsoft的年相关系数：

In [68]:
pct_px.groupby(get_year).apply(lambda x :x['AAPL'].corr(x['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 10.4 透视表和交叉表
在Python和pandas中，可以通过本章所介绍的groupby功能以及（能够利用层次化索引的）重塑运算制作透视表。DataFrame有一个pivot_table方法，此外还有一个顶级的pandas.pivot_table函数。除能为groupby提供便利之外，pivot_table还可以添加分项小计，也叫做margins。
回到小费数据集，假设我想要根据day和smoker计算分组平均数（pivot_table的默认聚合类型），并将day和smoker放到行上：

In [70]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [72]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

现在，假设我们只想聚合tip_pct和size，而且想根据time进行分组。我将smoker放到列上，把day放到行上：

In [74]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

还可以对这个表作进一步的处理，传入margins=True添加分项小计。这将会添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计：

In [75]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

要使用其他的聚合函数，将其传给aggfunc即可。例如，使用count或len可以得到有关分组大小的交叉表（计数或频率）：

In [78]:
tips.pivot_table(['tip_pct'],index=['time','smoker'],columns='day',aggfunc=len,margins=True)

tip_pct                         
day               Fri   Sat   Sun  Thur    All
time   smoker                                 
Dinner No         3.0  45.0  57.0   1.0  106.0
       Yes        9.0  42.0  19.0   NaN   70.0
Lunch  No         1.0   NaN   NaN  44.0   45.0
       Yes        6.0   NaN   NaN  17.0   23.0
All              19.0  87.0  76.0  62.0  244.0

如果存在空的组合（也就是NA），你可能会希望设置一个fill_value：

In [79]:
tips.pivot_table(['tip_pct'],index=['time','smoker'],columns='day',aggfunc=len,margins=True,fill_value=0)

tip_pct                    
day               Fri Sat Sun Thur    All
time   smoker                            
Dinner No           3  45  57    1  106.0
       Yes          9  42  19    0   70.0
Lunch  No           1   0   0   44   45.0
       Yes          6   0   0   17   23.0
All                19  87  76   62  244.0

## 交叉表：crosstab
交叉表（cross-tabulation，简称crosstab）是一种用于计算分组频率的特殊透视表。看下面的例子：

In [82]:
data=pd.DataFrame({'sample':np.arange(1,11),'nationality':['USA']*2+['Japan']*3+['USA']*2+['Japan']*3
                  ,'handedness':['Right-handed','Left-handed']*2+['Right-handed']*3+['Left-handed','Right-handed','Left-handed']})
data

,handedness,nationality,sample
0,Right-handed,USA,1
1,Left-handed,USA,2
2,Right-handed,Japan,3
3,Left-handed,Japan,4
4,Right-handed,Japan,5
5,Right-handed,USA,6
6,Right-handed,USA,7
7,Left-handed,Japan,8
8,Right-handed,Japan,9
9,Left-handed,Japan,10


作为调查分析的一部分，我们可能想要根据国籍和用手习惯对这段数据进行统计汇总。虽然可以用pivot_table实现该功能，但是pandas.crosstab函数会更方便：

In [83]:
pd.crosstab(data['nationality'],data['handedness'],margins=True)

handedness,Left-handed,Right-handed,All
nationality,,,
Japan,3,3,6
USA,1,3,4
All,4,6,10


In [84]:
data.pivot_table(['sample'],index=['nationality'],columns=['handedness'],aggfunc=len,margins=True)

sample                 
handedness  Left-handed Right-handed All
nationality                             
Japan                 3            3   6
USA                   1            3   4
All                   4            6  10

crosstab的前两个参数可以是数组或Series，或是数组列表。就像小费数据：

In [86]:
pd.crosstab([tips['time'],tips['day']],tips['smoker'],margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244